In [250]:
import pandas as pd
import numpy as np
from sklearn import metrics
from data_analysis import get_start, get_start_end

In [251]:
filepath = "../Data/Combined/FHD_55.csv"
data = pd.read_csv(filepath, index_col=False)

In [252]:
#for val in data:
#    if val != 'time':
#        data[val] = (data[val] - min(data[val])) / (max(data[val]) - min(data[val]))

In [253]:
data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)

In [254]:
idx_max = -1
mx = -float('inf')
for i in range(len(data['controller_right_vel'])):
    if float(data['controller_right_vel'][i]) > mx:
        mx = float(data['controller_right_vel'][i])
        idx_max = i

In [255]:
mx, idx_max

(10.075528058846395, 103)

In [256]:
filepath = "../Data/Combined/SRV_50.csv"
data = pd.read_csv(filepath, index_col=False)

data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)
data['controller_right_rel_headset'] = (data['headset_pos.z']) - (data['controller_right_pos.z'])


idx_max = -1
mx = -float('inf')
for i in range(len(data['controller_right_vel'])):
    if float(data['controller_right_vel'][i]) > mx:
        mx = float(data['controller_right_vel'][i])
        idx_max = i

print(data['controller_right_vel.y'][idx_max])
print(data['controller_right_vel.x'][idx_max])
print(data['controller_left_vel.z'][idx_max])

-11.74106
0.1610769
-2.119427


In [257]:
def simple_stat_classifier(filepath):
    data = pd.read_csv(filepath, index_col=False)
    data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)

    idx_max = -1
    max = -float('inf')
    for i in range(len(data['controller_right_vel'])):
        if float(data['controller_right_vel'][i]) > max:
            max = float(data['controller_right_vel'][i])
            idx_max = i

    if float(data['controller_right_vel.y'][idx_max]) > 0:
        # BHD or FHD
        if float(data['controller_right_vel.x'][idx_max]) < 0:
            return "FHD"
        else:
            return "BHD"
    else:
        # SRV or VOL
        if float(data['controller_right_vel.y'][idx_max]) < -3.5: # Really wouldn't want to use this part...
            return "SRV"
        else:
            return "VOL"

In [258]:
simple_stat_classifier("../Data/Combined/BHD_13.csv")

'BHD'

Preliminary Data Analysis seems to show that if we just take the controller_right_vel.y datapoint at the highest controller velocity, we can differentiate between serves and FHD/BHD/VOL. 

SRV is highly negative. (controller moving downwards)

FHD, BHD, are highly positive. (controller moving upwards)

VOL is slightly negative. (controller moving downwards)

Then out of FHD, BHD, we can check controller_right_vel.x
 - If > 0, BHD. (Controller moving to the right)
 - If < 0, FHD. (Controller moving to the left)

But magnitude is dependent on person, so we might not be able to use that... Might only be able to use +/-

In [259]:
def test_accuracy(classifier, errors=False):
    """
    Function to test how accurate your classifier is. Takes a classifier 
    (a function) that takes a filepath, and returns a string, either 
    'BHD', 'FHD', 'VOL', or 'SRV'. 

    Also can specifiy whether to print errors.

    Returns accuracy score of the classifier. 
    """
    actual = []
    predicted = []
    csv_number = []
    for action in ['BHD','FHD','VOL','SRV']:
        for i in range(1,91):
            filepath = "../Data/Combined/" + action + "_" + str(i).zfill(2) + ".csv"
            actual.append(action)
            predicted.append(classifier(filepath))
            csv_number.append(i)
    
    if errors:
        for i in range(len(actual)):
            if actual[i] != predicted[i]:
                print(actual[i], predicted[i], csv_number[i])
    
    return metrics.accuracy_score(actual, predicted)

In [260]:
def simple_stat_classifier_2(filepath):
    """
    Takes a filepath containing a sensor trace. First trims data according to
    isolate the swing action. Then, based on trimmed csv file, runs statistical
    classifier to determine swing type.

    Returns string, determining guessed swing type.
    """
    data = pd.read_csv(filepath, index_col=False)
    data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)

    start, end = get_start_end(data) # Determine start and end of swing
    data = data[start:end]
    
    idx_max = -1
    mx = -float('inf') # Finding moment of maximum velocity
    for i in range(len(data['controller_right_vel'])):
        i += start
        if float(data['controller_right_vel'][i]) > mx:
            mx = float(data['controller_right_vel'][i])
            idx_max = i

    if float(data['controller_right_vel.y'][idx_max]) > 0: # If swing moves upwards
        # BHD or FHD
        if float(data['controller_right_vel.x'][idx_max]) < 0: # If swing moves left/right
            return "FHD"
        else:
            return "BHD"
    else:
        # SRV or VOL    # Finding peak of controller, relative to headset
        if max((data['controller_right_pos.y'] - data['headset_pos.y'])) > 0.2: # 0.2 may need to be normalized
        #if max((data['controller_right_pos.y'] - data['headset_pos.y'])) > 0: # This works for Ian/Lucien, but gets Jason wrong every single time lol
            return "SRV"
        else:
            return "VOL"

TESTING BELOW

In [261]:
filepath = "../Data/Combined/SRV_57.csv"
data = pd.read_csv(filepath, index_col=False)

data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)

idx_max = -1
mx = -float('inf')
for i in range(len(data['controller_right_vel'])):
    if float(data['controller_right_vel'][i]) > mx:
        mx = float(data['controller_right_vel'][i])
        idx_max = i

In [262]:
s = get_start(data)
data['headset_pos.y'][s] - data['controller_right_pos.y'][s]

-0.40462167

In [263]:
data['headset_pos.y'].mean()

0.015195051647058843

In [264]:
test_accuracy(simple_stat_classifier)

0.9805555555555555

In [265]:
test_accuracy(simple_stat_classifier_2)

0.9944444444444445